In [1]:
import pandas as pd 
import numpy as np


In [2]:
data = pd.read_csv('hadoop.csv')


In [3]:
data = data.drop(['idx','access_date','access_time','access_page','access_referer','item_name'],axis=1)

In [4]:
unique_ip = data[data['item_type']=='cart']['access_ip'].unique()

## Make Dict_ip insert key is ip item is values

In [5]:
dict_ip = dict.fromkeys(unique_ip)   #dict key 생성

for i in dict_ip.keys():
    dict_ip[i] = []

#-----------------------------------------------#

for i in range(len(data)):          #dict value 삽입
    if (data['access_ip'].iloc[i] in dict_ip.keys()):
        key = data['access_ip'].iloc[i]
        dict_ip[key].append(data['item_id'].iloc[i])

In [6]:
for i,j in dict_ip.items():
    dict_ip[i] = list(map(str,j))        #dict value를 string으로 매핑
    
#------------------------------------------------#
#dict의 벨류가 4 미만이면 삭제
dict_ip = {key:value for key, value in dict_ip.items() if len(value) >4} 

In [7]:
cart_item = []       #dict의 마지막 아이템을 카트 아이템배열에 넣음
for i,j in dict_ip.items():
    cart_item.append(j[-1])

In [8]:
columns_name = []      #column이름을 string으로 변환
for i in range(14):
    columns_name.append(str(i))

## Make DataFrame New

In [78]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.471253,-0.534887,-0.544340,-0.534887,-0.522199,-0.534887,-0.527233,-0.544340,-0.522199,-0.522199,-0.534887,-0.527233,-0.522199,-0.522199
1,-0.291721,-0.365402,-0.365402,-0.365989,-0.365402,-0.348770,-0.365989,-0.365989,-0.348770,-0.348770,-0.348770,-0.348770,-0.365402,-0.365402
2,-4.510532,-1.671708,-1.325030,-1.850459,-2.837550,-1.597191,-2.165611,-1.629844,-2.234840,-1.562182,-1.783685,-1.656702,-1.863676,-1.863676
3,-4.510532,-1.647365,-1.627058,-1.690243,-1.579481,-1.092230,-3.945206,-3.945206,-1.492767,-1.398932,-3.821677,-1.901273,-1.625965,-1.625965
4,-0.895579,-1.005187,-1.005187,-0.999168,-0.992357,-1.000765,-1.000765,-0.985653,-0.993855,-0.981220,-0.992357,-0.985653,-1.005187,-1.005187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6963,-3.945206,-3.945206,-4.140063,-4.140063,-2.271880,-3.684368,-0.522199,-0.522199,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.522199
6964,-1.719576,-1.937196,-1.740164,-2.093109,-1.638971,-1.638971,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-1.638971
6965,-4.789981,-1.820540,0.105234,-1.372041,-1.468511,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-1.468511
6966,-0.365989,-0.365989,-0.365989,-0.544340,-0.348770,-0.544340,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.544340


In [9]:
df = pd.DataFrame.from_dict(dict_ip,orient='index',columns=columns_name)
df['13'] = cart_item

In [10]:
mask = df.applymap(lambda x: x is None)         #None값들 삭제
cols = df.columns[(mask).any()]
for col in df[cols]:
    df.loc[mask[col], col] = ''


In [11]:
item_list = []                 #item들을 뽑아서 한 row씩 한 리스트로 묶어 배열 생성
for i in range(len(df)):
    item_list.append(df.iloc[i,:].tolist())

# Word2Vec

In [12]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

In [13]:
path = get_tmpfile('word2vec.model')

In [14]:
model = Word2Vec(item_list, size=1, window=5, min_count=1,workers=4)
model.save('word2vec.model')

In [15]:
model = Word2Vec.load('word2vec.model')

In [16]:
model.train([['0','12']],total_examples=1 , epochs=1)

(0, 2)

In [17]:
word_vectors = model.wv  
vocabs = word_vectors.vocab.keys() #vocabs has keys that is item number

#word_vectors_list has values that is vector value from item number
word_vectors_list = [word_vectors[v] for v in vocabs] 


In [18]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [19]:
# def plot_2d_graph(vocabs, xs, ys):
#     plt.figure(figsize=(12,6))
#     plt.scatter(xs,ys,marker='o')
#     for i, v in enumerate(vocabs):
#         plt.annotate(v,xy=(xs[i],ys[i]))

In [20]:
#pca = PCA(n_components=2)

In [21]:
# xys = pca.fit_transform(word_vectors_list)
# xs = xys[0]
# ys = xys[1]

In [22]:
# plot_2d_graph(vocabs,xs,ys)

In [23]:
for i in range(len(df)):            #word2vec value를 df로 넣음
    for j in range(len(df.iloc[i])):
        key=str(df.iloc[i,j])
        df.iloc[i,j] = word_vectors[key].tolist()[0]

In [24]:
df.index = range(len(df))

In [95]:
df = df.apply(pd.to_numeric)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.471253,-0.534887,-0.544340,-0.534887,-0.522199,-0.534887,-0.527233,-0.544340,-0.522199,-0.522199,-0.534887,-0.527233,-0.522199,-0.522199
1,-0.291721,-0.365402,-0.365402,-0.365989,-0.365402,-0.348770,-0.365989,-0.365989,-0.348770,-0.348770,-0.348770,-0.348770,-0.365402,-0.365402
2,-4.510532,-1.671708,-1.325030,-1.850459,-2.837550,-1.597191,-2.165611,-1.629844,-2.234840,-1.562182,-1.783685,-1.656702,-1.863676,-1.863676
3,-4.510532,-1.647365,-1.627058,-1.690243,-1.579481,-1.092230,-3.945206,-3.945206,-1.492767,-1.398932,-3.821677,-1.901273,-1.625965,-1.625965
4,-0.895579,-1.005187,-1.005187,-0.999168,-0.992357,-1.000765,-1.000765,-0.985653,-0.993855,-0.981220,-0.992357,-0.985653,-1.005187,-1.005187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6963,-3.945206,-3.945206,-4.140063,-4.140063,-2.271880,-3.684368,-0.522199,-0.522199,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.522199
6964,-1.719576,-1.937196,-1.740164,-2.093109,-1.638971,-1.638971,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-1.638971
6965,-4.789981,-1.820540,0.105234,-1.372041,-1.468511,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-1.468511
6966,-0.365989,-0.365989,-0.365989,-0.544340,-0.348770,-0.544340,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.893878,-0.544340


# Data split && Learning

In [26]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [27]:
X_data = df.drop('13',axis=1)
y_data = df['13']

In [28]:
seed = 7
test_size = 0.3
X_train,X_test,y_train,y_test = train_test_split(X_data,y_data,test_size=test_size, random_state=seed)

In [29]:
model = xgb.XGBClassifier()

In [30]:
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.5,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [31]:
result = pd.DataFrame()

In [32]:
prediction = model.predict(X_test)

In [33]:
result['X_test'] = prediction
result['Y_test'] = y_test.tolist()

In [90]:
from IPython.display import display
result = np.transpose(result)
pd.options.display.max_columns = None
display(result)

array(None, dtype=object)

In [35]:
X_train.index = range(len(X_train))

In [36]:
y_train.index = range(len(y_train))

In [41]:
from sklearn import metrics

In [42]:
# confusion_matrix(prediction,y_test)

# Test

In [63]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [75]:
pca = PCA(n_components=1)
xys = pca.fit_transform(word_vectors_list)
xs = xys[:,0]

In [94]:
y_data

0      -0.522199
1      -0.365402
2      -1.863676
3      -1.625965
4      -1.005187
          ...   
6963   -0.522199
6964   -1.638971
6965   -1.468511
6966   -0.544340
6967   -1.021930
Name: 13, Length: 6968, dtype: float64

In [80]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.291721,-0.348770,-0.365989,-0.348770,-0.365402,-0.365989,-0.348770,-0.365989,-0.365989,-0.365402,-0.348770,-0.365989,-0.365989
1,-0.895579,-0.985653,-1.000765,-0.992357,-1.005187,-0.985653,-0.981220,-0.992357,-0.999168,-1.000765,-0.985653,-1.005187,-0.981220
2,-0.940116,-1.191396,-1.191396,-1.021930,-1.045920,-1.191396,-1.059488,-1.048728,-1.050005,-1.045920,-1.021930,-1.059488,-1.059488
3,-0.940116,-1.059488,-1.021930,-1.048728,-1.059488,-1.059488,-1.021930,-1.045920,-1.021930,-1.053053,-1.050005,-1.050005,-1.191396
4,-4.789981,-1.412159,-3.945206,-1.650466,-3.684368,-1.268069,-1.528863,-4.319551,-1.498557,-1.574312,-0.892759,-1.191396,-1.152024
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4872,-0.048665,-0.125691,-0.125691,-0.118668,-0.125691,-0.125691,-0.125691,-0.125691,-0.125691,-0.118668,-0.125691,-0.125691,-0.118668
4873,-0.471253,-0.544340,-0.544340,-0.544340,-0.544340,-0.534887,-0.534887,-0.544340,-0.534887,-0.534887,-0.534887,-0.534887,-0.527233
4874,-0.895579,-0.999168,-0.992357,-0.992357,-0.993855,-0.999168,-0.999168,-0.993855,-0.993855,-0.999168,-0.993855,-1.005187,-1.000765
4875,-4.510532,-2.039547,-3.821677,-1.863135,-2.272730,-1.468608,-1.425416,-3.821677,-1.868956,-3.945206,-1.587632,-4.319551,-1.458987
